In [2]:
import pandas as pd
import sys
import glancewriterDecoder as gd
import re
sys.path.append("..") #voodoo shit
from trie.keyboard import create_keyboard
from trie.trie import Node, insert_key
from trie.predict import predict

from clustering.TCluster import TCluster

import os
import json
%matplotlib widget

In [26]:
def GetGazePathsFromFile(file_path):
    try:
        # Open and read the file
        gazePaths = []
        topWords = []
        with open(file_path, 'r') as file:
            content = file.readlines()
        for line in content:
            # print(line)
            if (line.strip().startswith('{')):
                corrected_content = correct_json(line)
                # print(corrected_content)
                data = json.loads(corrected_content)
                if "top_words" in data:
                    topWords.append(data["top_words"])
                if "gaze_points" in data:
                    points = data["gaze_points"]
                    points = [(point['x'], point['y'], point['z']) for point in points]
                    gazePaths.append(points)
        return gazePaths, topWords
        # for i, path in enumerate(gazePaths):
        #     print(i)
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return None
    except json.JSONDecodeError as e:
        print(f"Error: Invalid JSON format in the file: {str(e)}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None

def correct_json(content):
    # Replace single quotes with double quotes
    content = content.replace("'", '"')
    
    # Correct boolean values
    content = content.replace('True', 'true').replace('False', 'false')
    
    # Remove trailing commas in lists and objects
    content = re.sub(r',\s*}', '}', content)
    content = re.sub(r',\s*]', ']', content)
    
    # Wrap the entire content in curly braces if it's not already
    if not content.strip().startswith('{'):
        content = '{' + content + '}'
    
    return content

In [27]:
gaze_paths, top_words = GetGazePathsFromFile("../eyeData/eyeTracking2024-10-02 15-48-10.txt")

In [5]:

keyboard_layout = gd.read_keyboard_layout("circleLayout.txt")
decoder = gd.GlanceWriterDecoder(keyboard_layout)

# Insert words into the trie
# words = ["move", "more", "mode", "mole"]
df_training = pd.read_csv('../data/vocab_final.csv')

training_words = df_training['word'].tolist()

# Filter only the words that are alpha
training_words = [str(word).lower() for word in training_words if str(word).isalpha()]

for word in training_words:
    decoder.insert_word(word)

# Simulate a gaze path (this should be replaced with actual gaze data)
for i in range(1):
    gaze_path = gaze_paths[i]
    gaze_path = [(point[0], point[1]) for point in gaze_path]
    print(gaze_path)
    # with open("sample_path.txt", "r") as file:
    #     for line in file:
    #         noParen = line[1:-2]
    #         nums = noParen.split(",")
    #         gaze_path.append((float(nums[0].strip()), float(nums[1].strip())))
    # gaze_path = [(85, 55), (95, 45), (130, 15), (180, 35)]

    # Decode the gaze path
    result = decoder.decode_gaze_path(gaze_path)
    print("Top 5 candidates:", result)

[(-0.34633731842041016, -0.04733343422412872), (-0.43738827109336853, -0.023766815662384033), (-0.39010536670684814, -0.03548897057771683), (-0.3468334674835205, -0.045335568487644196), (-0.30884596705436707, -0.04058998078107834), (-0.26352375745773315, -0.030061714351177216), (-0.27201828360557556, -0.0325019508600235), (-0.2778249979019165, -0.01756148785352707), (-0.2924300730228424, -0.03718181699514389), (-0.3109557330608368, -0.03505685180425644), (-0.2716694176197052, -0.037626639008522034), (-0.3100527822971344, -0.015218585729598999), (-0.2829872965812683, -0.03866095840930939), (-0.2997112274169922, -0.02582801878452301), (-0.3973274528980255, -0.020122364163398743), (-0.3909473717212677, 0.016609027981758118), (-0.3826747536659241, 0.019045390188694), (-0.3454795479774475, 0.00588805228471756), (-0.4309781789779663, 0.00205964595079422), (-0.45234251022338867, 0.03178364410996437), (-0.4536591172218323, 0.03692755103111267), (-0.4630325436592102, 0.030335139483213425), (-0.

In [56]:
vocab_path = os.path.join('../', 'data', 'vocab_final.csv')
vocab = pd.read_csv(vocab_path)

# Filter only the words that are alpha
training_words = [str(word).lower() for word in training_words if str(word).isalpha()]
root = Node()
for word in training_words:
    insert_key(root, word)



with open("circleLayout.txt", "r") as f:
    text = f.read()
    corrected_text = correct_json(text)
    layout = json.loads(corrected_text)
    custom_keyboard = create_keyboard(layout["keyboard"], useString=True)
    center = (layout['center']['x'], layout['center']['y'])
    inner_radius = layout['inner_radius']
    outer_radius = layout['outer_radius']

print(center)
print(inner_radius)
print(outer_radius)
bigram_path = os.path.join('..', 'data', 'bigram_v2.json')
truth = ["the", "leaf", "drops", "drops", "to", "the", "ground", "", "the", "night", "sky", "is",
 "bright", "and", "and", "clear", "clear", "a", "small", "stream", "stream", "flows", "under", "the", "bridge",
  "gentle", "waves", "lap", "at", "at", "the", "shore", "everyone", "everyone", "must", "know", "all", "the", "rules", "", "", ""]
with open(bigram_path, 'r') as f:
    bigram_probs: dict[dict] = json.load(f)


for i in range(len(top_words)):
    # gaze_path = gaze_paths[i]
    last_two: list[str] = ['<s>', '<s>']
    context_probs: dict[dict] = bigram_probs.get(' '.join(last_two), {})
    tc = TCluster(K=1, eps=0.07, vocab=vocab, context_probs=context_probs)

    filtered_gaze_path = gaze_paths[i]
    filtered_gaze_path = [(point[0], point[1], point[2]) for point in filtered_gaze_path if ((point[0] - center[0])**2 + (point[1] - center[1])**2 > inner_radius**2 and 
                                                                        (point[0] - center[0])**2 + (point[1] - center[1])**2 < outer_radius**2)]
    df = pd.DataFrame(filtered_gaze_path, columns=['x', 'y', 'time'])
    
    tc.fit(df)
    
    keys = tc.predict(custom_keyboard, root)
    predictions = [key[0] for key in keys]
    if (truth[i] in predictions):
        score += 1
    # print(truth[i])
    # print(str(predictions))
    print("offline predicted words: " + str(predictions))
    print("words predicted live: " + str(top_words[i]))


(-0.009399980306625366, -0.125)
0.3230000138282776
0.5
offline predicted words: ['the', 'he', 'fee']
words predicted live: ['the', 'he']
offline predicted words: ['ledge', 'face', 'leaf']
words predicted live: ['ledge', 'leaf', 'face']
offline predicted words: ['stops', 'troops', 'crops']
words predicted live: ['stops', 'troops', 'crops']
offline predicted words: ['crops', 'drops', 'cops']
words predicted live: ['crops', 'drops', 'cops']
offline predicted words: ['too', 'up', 'to']
words predicted live: ['too', 'up', 'to']
offline predicted words: ['the', 'he', 'fee']
words predicted live: ['the', 'he', 'fee']
offline predicted words: ['ground', 'round', 'hound']
words predicted live: ['ground', 'round', 'hound']
offline predicted words: ['fee']
words predicted live: []
offline predicted words: ['the', 'he']
words predicted live: ['the', 'he']
offline predicted words: ['night', 'might', 'light']
words predicted live: ['night', 'light', 'might']
offline predicted words: ['giddy', 'sky',

In [ ]:
for k in range(20):
    score = 0
    eep = float(k+1)/100.0
    for i in range(len(top_words)):
        # gaze_path = gaze_paths[i]
        last_two: list[str] = ['<s>', '<s>']
        context_probs: dict[dict] = bigram_probs.get(' '.join(last_two), {})
        tc = TCluster(K=1, eps=eep, vocab=vocab, context_probs=context_probs)

        filtered_gaze_path = gaze_paths[i]
        filtered_gaze_path = [(point[0], point[1], point[2]) for point in filtered_gaze_path if ((point[0] - center[0])**2 + (point[1] - center[1])**2 > inner_radius**2 and 
                                                                            (point[0] - center[0])**2 + (point[1] - center[1])**2 < outer_radius**2)]
        df = pd.DataFrame(filtered_gaze_path, columns=['x', 'y', 'time'])
        
        tc.fit(df)
        
        keys = tc.predict(custom_keyboard, root)
        predictions = [key[0] for key in keys]
        if (truth[i] in predictions):
            score += 1
        # print(truth[i])
        # print(str(predictions))
        # print("offline predicted words: " + str(predictions))
        # print("words predicted live: " + str(top_words[i]))
    print(score)
    print(eep)